# 🧠 Writers Room Notebook: Concept Generator + Feedback Loop

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load from .env file

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import json
from pathlib import Path
from uuid import uuid4
from IPython.display import display, Markdown

In [7]:
# === Configuration ===
MEMORY_FILE = Path("writers_room_memory.json")

In [8]:
# === Memory Backend ===
def init_memory():
    if not MEMORY_FILE.exists():
        with open(MEMORY_FILE, "w") as f:
            json.dump({}, f)

def save_to_memory(table: str, key: str, content):
    init_memory()
    with open(MEMORY_FILE, "r+") as f:
        db = json.load(f)
        db.setdefault(table, {})[key] = content
        f.seek(0)
        json.dump(db, f, indent=2)
        f.truncate()

def get_from_memory(table: str, key: str):
    init_memory()
    with open(MEMORY_FILE, "r") as f:
        db = json.load(f)
    return db.get(table, {}).get(key, None)

In [9]:
def export_memory(file_path: str):
    init_memory()
    with open(MEMORY_FILE, "r") as src:
        with open(file_path, "w") as dst:
            dst.write(src.read())

def import_memory(file_path: str):
    with open(file_path, "r") as src:
        with open(MEMORY_FILE, "w") as dst:
            dst.write(src.read())

In [10]:
# === UI Helpers ===
def display_output(text):
    display(Markdown(f"### Output:\n{text}"))

def choose_from_versions(versions):
    for i, v in enumerate(versions):
        display(Markdown(f"**Version {i+1}:**\n{v}"))
    index = int(input("Choose final version (number): "))
    return versions[index - 1]

In [11]:
# === Feedback Loop Utility ===
def feedback_loop(component_name, generate_func, revise_func, prompt):
    versions = []
    output = generate_func(prompt)
    versions.append(output)
    display_output(output)

    while True:
        feedback = input("\nFeedback? Press Enter to accept, or type your comments: ")
        if not feedback:
            break
        output = revise_func(versions[-1], feedback)
        versions.append(output)
        display_output(output)

    final = choose_from_versions(versions)
    save_to_memory("concept", component_name, final)
    print(f"\n Final version saved to memory under 'concept' → '{component_name}'")
    return final

In [12]:
# === Mock LLM Integration ===
def generate_show_thesis(prompt):
    return f"A gripping teen drama set in {prompt}, where ambition collides with loyalty."

def revise_with_feedback(previous_output, feedback):
    return previous_output + f" (Revised with note: '{feedback}')"

In [13]:
# === Try It Out ===
prompt = "an elite ski racing school in Aspen"
final_thesis = feedback_loop("show_thesis", generate_show_thesis, revise_with_feedback, prompt)

In [ ]:
# === Optional: Export/Import Memory ===
# export_memory("backup_writers_room.json")
# import_memory("backup_writers_room.json")